## ⚠THIS NOTEBOOK WILL NOT AUTO-SAVE⚠

# ⌚Demo Time⌚

Let's get started with Python by loading the package for Q# interoperability called `qsharp`.

In [ ]:
import qsharp
qsharp.component_versions()

# Task: Generate _quantum_ random numbers

We want to make a truly* random source that can generate a list of random bits like this:

In [ ]:
randomness = [0,1,0,0,1,1,0,1,0,1]

<br><br>
<tiny>*still simulated here so still pseudo-random</tiny>

## Generating _quantum_ random numbers with Q\# ##

```Q#
    operation Qrng() : Result {
        use qubit = Qubit();   // Preparing the qubit
        H(qubit);               // Do operation H
        return MResetZ(qubit);  // Measure and reset qubit
    }
```
How can we dive in to what is going on here?

### Let's load the Q# code from Python!

In [ ]:
from OSD.Demo import Qrng

In [ ]:
directQRNG = qsharp.compile("""
    open Microsoft.Quantum.Measurement;
    operation Qrng() : Result {
        use qubit = Qubit();   // Preparing the qubit
        H(qubit);               // Do operation H
        return MResetZ(qubit);  // Measure and reset qubit
    }
""")

## Understanding `SampleQrng`

We can use built-in documentation strings, just like we can with Python functions.

In [ ]:
?Qrng

That tells us what we can **do** with `SampleQrng`:

In [ ]:
[Qrng.simulate() for _ in range(10)]

## ✨**Your turn!** ✨

Write a python function that will generate a random number 0-15

In [ ]:
def random_quantum_int():
    bin_string = "".join(str(elem) for elem in [Qrng.simulate() for _ in range(4)])
    return int(bin_string, 2)

In [ ]:
random_quantum_int()

## Hold up: What is a qubit?

- Answer: a single unit of information in a quantum computer 
    - _quantum + bit = qubit_

- We can predict what a single qubit will do by using a column vector of 2 complex numbers* like this:

$\left|{x}\right\rangle = \left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

We say that this vector is the representation of the **state** our qubit has.

Using quantum states, you can predict and simulate how quantum computers work!

## What can we _do_ with a qubit?

Similar to classical bits on your computer, you can do three types of things with qubits:

- Prepare a qubit
- Do operations with a qubit
- Measure a qubit, getting classical data back (e.g.: 0 or 1)



```
namespace OSD.Demo {
    operation Qrng() : Result {
        use qubit = Qubit();  // Preparation
        H(qubit);             // Operation 
        return M(qubit);      // Measurement
    }
}
```

### How can we "get" a qubit?

In [ ]:
prepare_qubit = qsharp.compile("""
open Microsoft.Quantum.Diagnostics;

operation PrepareQubit() : Unit {
    use qubit = Qubit();     // We want 1 qubit to use for our task
        DumpMachine();     // Print out what the simulator is keeping a record of
}
""")

The `DumpMachine` call asks the simulator to print the state that it's using to simulate your quantum program.

In [ ]:
prepare_qubit.simulate()

You can read the above output like the vector we wrote above, where the first column is the index, the second is the real part of the vector at that position, and the second is the complex part of that vector entry.

What does `DumpMachine` tell us?

```
|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖
```

This is the same state we saw earlier!

$\left|{x}\right\rangle = \left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

# How about more qubits?!



<figure style="text-align: center;">
    <img src="https://disneygenderevolution.files.wordpress.com/2014/12/ariel-the-little-mermaid-i-want-more-gif.gif" width="60%">
    <caption>
      <br>  
        <strong></strong>
    </caption>
</figure>

## ✨**Your turn!** ✨

What would the representation of a 3 qubit state look like? A 4 qubit state? How does this scale?

In [ ]:
prepare_register = qsharp.compile("""
open Microsoft.Quantum.Diagnostics;

operation PrepareQubit() : Unit {
    use register = Qubit[3];     // We want 1 qubit to use for our task
        DumpMachine();     // Print out what the simulator is keeping a record of
}
""")

In [ ]:
prepare_register.simulate()

## Learning superposition by inspection

We can use `DumpMachine` again to understand see what the `H` operation does to our qubit.

In [ ]:
from OSD.Demo import QrngWithDiagnostics

QrngWithDiagnostics.simulate()

The operation `H` on our qubit puts our simulated qubit in **superposition**:

```
After using H(qubit) to create a superposition state:
|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖   
```

#### 🚨Note: `DumpMachine` is showing the information the simulator has!🚨

## Operations with multiple qubits can create 💕entanglement💕

Using Q# with Python, we can also explore other quantum effects that you can use in your programs, like **entanglement**.

```
operation EntangleQubits() : (Result, Result) {
    // Preparing two qubits
    use (qubit1, qubit2) = (Qubit(), Qubit());

    // The operations on the qubits needed to entangle them
    H(qubit1);
    CNOT(qubit1, qubit2);

    // Finally, measure and reset the qubits
    return (MResetZ(qubit1), MResetZ(qubit2));
}
```

In [ ]:
from OSD.Demo import EntangleQubits
results = EntangleQubits.simulate(verbose=True)

What does `DumpRegister` tell us this time?
```
|00⟩	0.7071067811865476 + 0𝑖
|01⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0.7071067811865476 + 0𝑖
```
- ∣00❭➡ measuring both qubits give you `(0, 0)`
- ∣11❭➡ measuring both qubits give you `(1, 1)`

Using this state to predict measurement results, we don't know if we'll get `(0, 0)` or `(1, 1)`, but we can rule out `(0, 1)` and `(1, 0)`.

No matter how many times we run, both measurements are equal to each other!

## Share the randomness

- If you **entangle** two qubits and then share one, then you both measure you will have the same random number*.

> If you are interested in this, check out quantum cryptography protocols like BB84!


In [ ]:
[EntangleQubits.simulate(verbose=False) for _ in range(10)]

## ✨**Your turn!** ✨

Can you tell the difference when measuring the entangled state you saw above prepared by:
```
H(qubit1);
CNOT(qubit1, qubit2);
```
from another entangled state by using `MResetZ`?
```
X(qubit1);
H(qubit1);
CNOT(qubit1, qubit2);
```
What could you change to distinguish these?
> The [Q# library docs](https://docs.microsoft.com/en-us/qsharp/api/qsharp/) could be helpful ♥

# 🏆BONUS🏆

## Building up quantum algorithm: Deutsch–Jozsa 

1. **Nimue**💃 asks **Merlin**🧙‍♂️ a single question of the form "Should _`heir`_ be the king?"
2. **Merlin**🧙‍♂️ must respond with either "yes" or "no," revealing nothing else.

### Merlin's possible strategies: a black box

| | `heir` = Arthur | `heir` = Mordred | |
|---|---|---|---|
| Pick Arthur⚔ | yes | no | ☑ |
| Pick Mordred🛡 | no | yes | ☑ |
| Pick both | yes | yes | ☒ |
| Pick neither | no | no | ☒ |

- _Inputs_ ➡ **Nimue's**💃 question
- _Outputs_ ➡ **Merlin's**🧙‍♂️ response
<br>
<br>
<figure style="text-align: left;">
    <img src="media/twobit.png" width="50%">
    <caption>
      <br>  
        <strong>Diagram of all possible one bit functions</strong>
    </caption>
</figure>

<figure style="text-align: left;">
    <img src="media/twobitDJ.png" width="40%">
    <caption>
      <br>  
        <strong>Global property of the one bit functions: Constant or Balanced</strong>
    </caption>
</figure>

> ### Deutsch–Jozsa Algorithim ###
>
>* **GIVEN:** A black box quantum operation (called an *oracle*) which acts on an input qubit and a target qubit.
>  We are promised that the oracle is either _constant_ or _balanced_. 
>					
>* **GOAL:** to determine if the oracle is _constant_ or _balanced_.

### ❕ Deutsch–Jozsa can solve the puzzle in **one** query to a black box❕ (even with more than 1 bit)

**Nimue**💃 is the lady of the lake, and hence has the power of quantum computing at her disposal.

She can use _Deutsch-Jozsa_ to test **Merlin**🧙‍♂️ without meddling in the affairs of mortals!

```
operation CheckIfOracleIsBalanced(oracle : ((Qubit, Qubit) => Unit)) : Bool {
    use (control, target) = (Qubit(), Qubit()));
    // Prepare superposition on the control register.
    H(control);                                   

    // Use the phase kickback technique to learn a global property of our oracle.
    within {
        X(target);
        H(target);
    } apply {
        oracle(control, target);
    }

    return MResetX(control) == One;                
}
```

## Let's run this Q\# code and see what it does...

In [ ]:
is_zero_oracle_balanced = qsharp.compile("""
    open OSD.DeutschJozsa;
    operation IsZeroOracleBalanced() : Bool {
    
        return CheckIfOracleIsBalanced(true, ApplyZeroOracle);
    }
""")

In [ ]:
is_zero_oracle_balanced.simulate()

Doing the same thing with the `one` oracle:

In [ ]:
qsharp.compile("""
    open OSD.DeutschJozsa;

    operation IsOneOracleBalanced() : Bool {
        return CheckIfOracleIsBalanced(true, ApplyOneOracle);
    }
""").simulate()

**NOTE:** The output states for the `zero` and `one` oracles differ only by a _global phase_ ; we can't tell which oracle we applied by looking at measurement results!

<figure style="text-align: left;">
    <img src="media/twobitDJ.png" width="40%">
    <caption>
      <br>  
        <strong>Global property of the one bit functions: Constant or Balanced</strong>
    </caption>
</figure>

On the other hand, if we apply a balanced oracle instead, what happens?

In [ ]:
is_not_oracle_balanced = qsharp.compile("""
    open OSD.DeutschJozsa;

    operation IsNotOracleBalanced(): Bool {
        return CheckIfOracleIsBalanced(true, ApplyNotOracle);
    }
""")

In [ ]:
is_not_oracle_balanced.simulate()

The sign in front of |00⟩ and |10⟩ changed, but |01⟩ and |11⟩ didn't, so it's not a _global_ phase, and we can measure it.

**Nimue**💃 can tell whether the oracle is constant or balanced, but not anything else; exactly what she wanted!

## Putting it all together: One query, one answer!

In [ ]:
from OSD.DeutschJozsa import RunDeutschJozsaAlgorithm

In [ ]:
RunDeutschJozsaAlgorithm.simulate(verbose=False)

In [ ]:
RunDeutschJozsaAlgorithm.simulate(verbose=True)

## Camelot's future is safe!

## You + Nimue💃 used a quantum algorithim to make sure of it 👍
<br>
<figure style="text-align: center;">
    <img src="https://thumbs.gfycat.com/SingleFatalAzurewingedmagpie-size_restricted.gif" width="60%">
    <caption>
      <br>  
        <strong></strong>
    </caption>
</figure>


## ... as long as Merlin🧙‍♂️ sticks to his role
<bR>
<figure style="text-align: center;">
    <img src="http://78.media.tumblr.com/tumblr_liogspF9bI1qh6b53o1_500.gif" width="60%">
    <caption>
      <br>  
        <strong></strong>
    </caption>
</figure>

# 📝 Review time 📝


- Make random numbers ✔
- Share random numbers with other people ✔
- Help Nimue make sure Camelot is on the right track ✔
- Use Python tools and skills to learn quantum computing ✔

---

## Helpful diagnostics :)

In [ ]:
for component, version in sorted(qsharp.component_versions().items(), key=lambda x: x[0]):
    print(f"{component:20}{version}")

In [ ]:
import sys
print(sys.version)